# member_finances (MF): Raw Data Discovery & Insights

### Target Attributes: member_id, monthly_fee, payment_status

In [ ]:
import sqlite3
import numpy as np
import matplotlib.pylab as plt
import pandas as pd
import seaborn as sns

conn = sqlite3.connect(r"__PATH__")

df = pd.read_sql_query("SELECT * FROM member_finances", conn)

### Data Understanding, Cleaning & Preparation

In [2]:
df.shape

(10240, 12)

In [3]:
df.head(2)

,finance_row_id,member_id,First Name,Last Name,monthly_fee,payment_status,Last Payment ID,Due Date,Notes,Payment Method,Discount Applied,Outstanding Balance
0,1,ABC1078,Mary,Leonard,$39.99,On-Time,PMT-00800,07/20/2024,Paid via auto-billing,Credit Card,Summer Promo,$0.00
1,2,ABC1179,Sarah,Odonnell,$39.99,On-Time,PMT-01806,02/02/2025,Paid via auto-billing,Credit Card,None,$0.00


In [4]:
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(" ", "_")
df.columns

Index(['finance_row_id', 'member_id', 'first_name', 'last_name', 'monthly_fee',
       'payment_status', 'last_payment_id', 'due_date', 'notes',
       'payment_method', 'discount_applied', 'outstanding_balance'],
      dtype='object')

In [5]:
df = df[[
    #'finance_row_id', 
    'member_id', 
    #'first_name', 'last_name', 
    'monthly_fee',
    #'payment_status', 
    #'last_payment_id', 
    'due_date', 
    # 'notes',
    #'payment_method', 'discount_applied', 
    'outstanding_balance'  
    ]].copy()

In [6]:
df.columns, df.dtypes

(Index(['member_id', 'monthly_fee', 'due_date', 'outstanding_balance'], dtype='object'),
 member_id              object
 monthly_fee            object
 due_date               object
 outstanding_balance    object
 dtype: object)

In [7]:
df.isna().sum()

member_id              0
monthly_fee            0
due_date               0
outstanding_balance    0
dtype: int64

In [8]:
df[df['member_id'] == 'ABC1000'].sort_values(by='due_date')

,member_id,monthly_fee,due_date,outstanding_balance
4765,ABC1000,$55.00,01/09/2025,$0.00
7890,ABC1000,$55.00,02/26/2025,$0.00
2997,ABC1000,$55.00,03/01/2025,$0.00
3937,ABC1000,$55.00,04/22/2025,$0.00
5019,ABC1000,$55.00,05/18/2025,$0.00
4255,ABC1000,$55.00,06/19/2025,$0.00
10053,ABC1000,$55.00,06/19/2025,$0.00
5569,ABC1000,$55.00,07/17/2025,$0.00
8640,ABC1000,$55.00,08/02/2025,$0.00
5717,ABC1000,$55.00,08/08/2024,$0.00


In [9]:
df['due_date'] = pd.to_datetime(df['due_date'], format='%m/%d/%Y', errors='coerce')

In [10]:
# Clean monthly_fee: remove currency symbols and convert to float
df['monthly_fee'] = df['monthly_fee'].replace('[$]', '', regex=True).astype(float)
df['outstanding_balance'] = df['outstanding_balance'].replace('[$]', '', regex=True).astype(float)

In [11]:
df['monthly_fee'].unique()

array([39.99, 59.99, 45.  , 55.  , 49.99])

In [12]:
df[df['member_id'] == 'ABC1000']

,member_id,monthly_fee,due_date,outstanding_balance
663,ABC1000,55.0,2024-12-11,0.0
1620,ABC1000,55.0,2024-11-15,0.0
1649,ABC1000,55.0,2024-10-17,0.0
2997,ABC1000,55.0,2025-03-01,0.0
3448,ABC1000,55.0,2024-09-17,0.0
3937,ABC1000,55.0,2025-04-22,0.0
4255,ABC1000,55.0,2025-06-19,0.0
4765,ABC1000,55.0,2025-01-09,0.0
5019,ABC1000,55.0,2025-05-18,0.0
5569,ABC1000,55.0,2025-07-17,0.0


In [13]:
# Create rule-based payment status from outstanding_balance
df['payment_status'] = np.where(df['outstanding_balance'] > 0, 'Late', 'On Time')
df[df['member_id'] == 'ABC1001']

,member_id,monthly_fee,due_date,outstanding_balance,payment_status
623,ABC1001,49.99,2024-03-09,0.00,On Time
962,ABC1001,49.99,2024-01-17,0.00,On Time
3975,ABC1001,49.99,2024-10-13,0.00,On Time
4555,ABC1001,49.99,2024-11-14,0.00,On Time
6092,ABC1001,49.99,2024-04-17,0.00,On Time
6598,ABC1001,49.99,2024-07-19,0.00,On Time
6600,ABC1001,49.99,2024-05-03,0.00,On Time
6870,ABC1001,49.99,2024-06-28,0.00,On Time
7131,ABC1001,49.99,2024-09-23,0.00,On Time
7647,ABC1001,49.99,2024-08-23,0.00,On Time


In [14]:
def majority_status(x):
    return 'late' if (x.str.lower().eq('late').mean() >= 0.5) else 'on-time'

payment_status_tendency = (
    df.groupby('member_id')['payment_status']
    .apply(majority_status)
    .reset_index(name='payment_status_tendency')
)

# merge back to table
df = df.merge(payment_status_tendency, on='member_id', how='left')


In [15]:
df[df['member_id'] == 'ABC1001'].sort_values(by='due_date')

,member_id,monthly_fee,due_date,outstanding_balance,payment_status,payment_status_tendency
962,ABC1001,49.99,2024-01-17,0.00,On Time,on-time
9276,ABC1001,49.99,2024-02-06,0.00,On Time,on-time
623,ABC1001,49.99,2024-03-09,0.00,On Time,on-time
6092,ABC1001,49.99,2024-04-17,0.00,On Time,on-time
6600,ABC1001,49.99,2024-05-03,0.00,On Time,on-time
6870,ABC1001,49.99,2024-06-28,0.00,On Time,on-time
6598,ABC1001,49.99,2024-07-19,0.00,On Time,on-time
7647,ABC1001,49.99,2024-08-23,0.00,On Time,on-time
7131,ABC1001,49.99,2024-09-23,0.00,On Time,on-time
3975,ABC1001,49.99,2024-10-13,0.00,On Time,on-time


In [16]:
df['total_payments'] = df.groupby('member_id')['member_id'].transform('size')

df[df['member_id'] == 'ABC1001']

,member_id,monthly_fee,due_date,outstanding_balance,payment_status,payment_status_tendency,total_payments
623,ABC1001,49.99,2024-03-09,0.00,On Time,on-time,12
962,ABC1001,49.99,2024-01-17,0.00,On Time,on-time,12
3975,ABC1001,49.99,2024-10-13,0.00,On Time,on-time,12
4555,ABC1001,49.99,2024-11-14,0.00,On Time,on-time,12
6092,ABC1001,49.99,2024-04-17,0.00,On Time,on-time,12
6598,ABC1001,49.99,2024-07-19,0.00,On Time,on-time,12
6600,ABC1001,49.99,2024-05-03,0.00,On Time,on-time,12
6870,ABC1001,49.99,2024-06-28,0.00,On Time,on-time,12
7131,ABC1001,49.99,2024-09-23,0.00,On Time,on-time,12
7647,ABC1001,49.99,2024-08-23,0.00,On Time,on-time,12


In [17]:
# last session date per member
last_session = (
    df.groupby('member_id')['due_date']
      .max()
      .reset_index(name='last_due_date')
)

lifetime_outstanding_balance = (
    df.groupby('member_id')['outstanding_balance']
      .sum()
      .reset_index(name='lifetime_outstanding_balance')
) 

# now build the member-level table
member_finances = (
    df[['member_id','monthly_fee','payment_status_tendency', 'total_payments']]
    .drop_duplicates('member_id', keep='last')
    .merge(lifetime_outstanding_balance, on='member_id', how='left')
)

member_finances[member_finances['member_id'] == 'ABC1001']

,member_id,monthly_fee,payment_status_tendency,total_payments,lifetime_outstanding_balance
467,ABC1001,49.99,on-time,12,49.99


In [18]:
member_finances.head(5)

,member_id,monthly_fee,payment_status_tendency,total_payments,lifetime_outstanding_balance
0,ABC1204,45.00,on-time,1,0.0
1,ABC1258,55.00,on-time,1,0.0
2,ABC1874,45.00,on-time,1,0.0
3,ABC1018,49.99,on-time,1,0.0
4,ABC1359,39.99,on-time,1,0.0


In [19]:
member_finances.shape

(1000, 5)

### Load to clean_db

In [ ]:
con_out = sqlite3.connect(r"__PATH__")
member_finances.to_sql("member_finances", con_out, if_exists="replace", index=False)
con_out.close()